<h1 align="center"><b> BUSN 32120 Final Project </b></h1>
<h2 align="center"> SQL Analysis on COVID–19 Fatality and Vaccination Rates </h2>

<div style="text-align: center;">
  <img src="covid-19.jpg" alt="COVID–19 Image" width="500"/>
</div>

<p align="center"><b>By: Bisma Rana</b></p>

## **Table of Contents**

- [Step 1: Import Libraries & Connect with SQLite Database](#step-1-import-libraries--connect-with-sqlite-database)
- [Step 2: Run SQL Query to Fetch Data](#step-2-run-sql-query-to-fetch-data)
- [Step 3: Run Queries of Analysis](#step-3-run-queries-of-analysis)
  - **Joins (x3)**
    - [Query 1: Join Cases and Vaccination Data by State and Date](#query-1-join-cases-and-vaccination-data-by-state-and-date)
    - [Query 2: Join State-Level New Cases with National Average New Cases](#query-2-join-state-level-new-cases-with-national-average-new-cases)
    - [Query 3: Join Booster Coverage Categories with Weekly Cases](#query-3-join-booster-coverage-categories-with-weekly-cases)
  - **Window Functions (x3)**
    - [Query 4: Window Function to Rank States by Weekly New Cases](#query-4-window-function-to-rank-states-by-weekly-new-cases)
    - [Query 5: Window Function to Determine National Week-over-Week % Change](#query-5-window-function-to-determine-national-week-over-week--change)
    - [Query 6: Window Function to Identify First Week with Reported COVID-19 Cases (> 0)](#query-6-window-function-to-identify-first-week-with-reported-covid-19-cases--0)
  - **Group By / Aggregation (x2)**
    - [Query 7: Group By State – Total and Average Weekly COVID-19 Metrics](#query-7-group-by-state--total-and-average-weekly-covid-19-metrics)
    - [Query 8: Group By Booster Coverage – Compare Average Cases and Deaths](#query-8-group-by-booster-coverage--compare-average-cases-and-deaths)
  - **Subqueries (x2)**
    - [Query 9: Subquery to Identify States with Low Vax and High Death Rates](#query-9-subquery-to-identify-states-with-low-vax-and-high-death-rates)
    - [Query 10: Subquery to Rank Top 5 States by Case Fatality Rate](#query-10-subquery-to-rank-top-5-states-by-case-fatality-rate)
  - **CTE (x1)**
    - [Query 11: CTE to Detect Spikes ≥ 50% in National New Cases](#query-11-cte-to-detect-spikes--50-in-national-new-cases)
  - **Stored Procedure (x1)**
    - [Query 12: Simulated Stored Procedure – Weekly Summary by State](#query-12-simulated-stored-procedure--weekly-summary-by-state)

### **Step 1: Import Libraries & Connect with SQLite Database**

In [3]:
# Import Libraries 
import sqlite3 # Enables connection and queries on SQLite databases
import pandas as pd # Allows loading and manipulating SQL results as DataFrames

# Connect to SQLite Database File 
conn = sqlite3.connect("covid_analysis.db")

### **Step 2: Run SQL Query to Fetch Data**

In [4]:
# Preview Cases & Deaths Sample 
query1 = "SELECT * FROM cases_deaths LIMIT 5;"   # Returns first 5 rows from cases_deaths table
df_cases_sample = pd.read_sql(query1, conn)
df_cases_sample

,state,start_date,date,total_cases,total_deaths,new_cases,new_deaths
0,LA,2020-01-16T00:00:00.000,2020-01-22 00:00:00,0.0,0.0,0.0,0.0
1,MN,2020-01-16T00:00:00.000,2020-01-22 00:00:00,0.0,0.0,0.0,0.0
2,SC,2020-01-16T00:00:00.000,2020-01-22 00:00:00,0.0,0.0,0.0,0.0
3,DE,2020-01-16T00:00:00.000,2020-01-22 00:00:00,5.0,0.0,5.0,0.0
4,IL,2020-01-16T00:00:00.000,2020-01-22 00:00:00,0.0,0.0,0.0,0.0


In [5]:
# Preview Vaccinations Sample 
query2 = "SELECT * FROM vaccinations_weekly LIMIT 5;"
df_vax_sample = pd.read_sql(query2, conn)
df_vax_sample

,state,date,fully_vaccinated_pct,one_dose_pct,booster_pct,fully_vaccinated_65plus_pct,booster_65plus_pct
0,AS,2020-12-09 00:00:00,0.0,0.0,0.0,0.0,0.0
1,LTC,2020-12-09 00:00:00,0.0,0.0,0.0,0.0,0.0
2,MP,2020-12-09 00:00:00,0.0,0.0,0.0,0.0,0.0
3,GU,2020-12-09 00:00:00,0.0,0.0,0.0,0.0,0.0
4,US,2020-12-09 00:00:00,0.0,0.0,0.0,0.0,0.0


### **Step 3: Run Queries of Analysis**

**Query 1: Join Cases and Vaccination Data by State and Date**

> **1. What is the goal of this query?**  
> To combine COVID–19 case and death data with vaccination percentages for each U.S. state by week, so they can be analyzed together.  

> **2. What does this query do?**  
> It performs an inner join between the `cases_deaths` and `vaccinations_weekly` tables using state and date as keys, and selects key columns related to new cases, deaths, and vaccination percentages.

> **3. What does the result show?**  
> The output displays a merged table of weekly COVID-19 case and death counts alongside vaccination percentages (fully vaccinated, booster, and one dose) for each U.S. state and date, allowing comparisons between infection rates and vaccination coverage over time.

In [11]:
query = """
-- QUERY 1: Join weekly cases with vaccination stats
SELECT
    c.state,
    c.date,
    c.new_cases,
    c.new_deaths,
    v.fully_vaccinated_pct,
    v.booster_pct,
    v.one_dose_pct
FROM cases_deaths c
JOIN vaccinations_weekly v
    ON c.state = v.state AND c.date = v.date
ORDER BY c.date, c.state;
"""

joined_df = pd.read_sql(query, conn)
joined_df

,state,date,new_cases,new_deaths,fully_vaccinated_pct,booster_pct,one_dose_pct
0,AS,2020-12-09 00:00:00,0.0,0.0,0.0,0.0,0.0
1,GU,2020-12-09 00:00:00,119.0,1.0,0.0,0.0,0.0
2,MP,2020-12-09 00:00:00,7.0,0.0,0.0,0.0,0.0
3,VI,2020-12-09 00:00:00,109.0,0.0,0.0,0.0,0.0
4,AK,2020-12-16 00:00:00,3417.0,30.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
7122,VT,2023-05-10 00:00:00,93.0,4.0,86.1,66.8,95.0
7123,WA,2023-05-10 00:00:00,1569.0,35.0,76.4,58.7,85.6
7124,WI,2023-05-10 00:00:00,1524.0,10.0,68.4,61.1,75.3
7125,WV,2023-05-10 00:00:00,194.0,8.0,59.8,49.6,67.6


**Query 2: Join State-Level New Cases with National Average New Cases**

> **1. What is the goal of this query?**  
> To see how each state’s new COVID-19 case count compares to the national average for the same date.

> **2. What does this query do?**  
> It first calculates the average number of new cases across all states for each date using a subquery, then joins that back to the state-level data to compute the difference from the national average.

> **3. What does the result show?**  
> The result shows how each state's new COVID-19 case count on a given date deviated from the national average, with states like Vermont and West Virginia reporting far fewer new cases than average, while Washington and Wisconsin exceeded the average, highlighting geographic variation in case trends.

In [10]:
query = """
-- QUERY 2: Join state-level new cases with national average new cases for the same date
SELECT 
    c.state,
    c.date,
    c.new_cases,
    avg_cases.avg_new_cases AS national_avg_cases,
    (c.new_cases - avg_cases.avg_new_cases) AS diff_from_avg
FROM cases_deaths c
JOIN (
    SELECT date, AVG(new_cases) AS avg_new_cases
    FROM cases_deaths
    GROUP BY date
) AS avg_cases
ON c.date = avg_cases.date
ORDER BY c.date, c.state;
"""

comparison_df = pd.read_sql(query, conn)
comparison_df

,state,date,new_cases,national_avg_cases,diff_from_avg
0,AK,2020-01-22 00:00:00,0.0,0.10,-0.10
1,AL,2020-01-22 00:00:00,0.0,0.10,-0.10
2,AR,2020-01-22 00:00:00,0.0,0.10,-0.10
3,AS,2020-01-22 00:00:00,0.0,0.10,-0.10
4,AZ,2020-01-22 00:00:00,0.0,0.10,-0.10
...,...,...,...,...,...
10375,VT,2023-05-10 00:00:00,93.0,1270.55,-1177.55
10376,WA,2023-05-10 00:00:00,1569.0,1270.55,298.45
10377,WI,2023-05-10 00:00:00,1524.0,1270.55,253.45
10378,WV,2023-05-10 00:00:00,194.0,1270.55,-1076.55


**Query 3: Join Booster Coverage Levels with Case Counts**

> **1. What is the goal of this query?**  
> To classify each state-week based on booster vaccination levels and compare it with new COVID-19 case counts.

> **2. What does this query do?**  
> It creates a CTE (vax_labels) that labels each state-date as having 'Low', 'Medium', or 'High' booster coverage using a CASE statement, then joins this with the cases_deaths table to analyze trends by coverage category.

> **3. What does the result show?**  
> The output lets us observe that states with high booster coverage (VT, WI) tend to report fewer new cases compared to those with low coverage (MP, VI), suggesting a potential protective effect associated with booster uptake.

In [13]:
query = """
-- QUERY 3: Join with categorized booster coverage labels
WITH vax_labels AS (
    SELECT
        state,
        date,
        booster_pct,
        CASE 
            WHEN booster_pct IS NULL THEN 'Unknown'
            WHEN booster_pct < 30 THEN 'Low'
            WHEN booster_pct < 60 THEN 'Medium'
            ELSE 'High'
        END AS booster_coverage_level
    FROM vaccinations_weekly
)
SELECT
    c.state,
    c.date,
    c.total_cases,
    c.new_cases,
    v.booster_pct,
    v.booster_coverage_level
FROM cases_deaths c
JOIN vax_labels v
    ON c.state = v.state AND c.date = v.date
ORDER BY c.date, c.state;
"""

labeled_df = pd.read_sql(query, conn)
labeled_df

,state,date,total_cases,new_cases,booster_pct,booster_coverage_level
0,AS,2020-12-09 00:00:00,0.0,0.0,0.0,Low
1,GU,2020-12-09 00:00:00,7039.0,119.0,0.0,Low
2,MP,2020-12-09 00:00:00,113.0,7.0,0.0,Low
3,VI,2020-12-09 00:00:00,1698.0,109.0,0.0,Low
4,AK,2020-12-16 00:00:00,41855.0,3417.0,0.0,Low
...,...,...,...,...,...,...
7122,VT,2023-05-10 00:00:00,154243.0,93.0,66.8,High
7123,WA,2023-05-10 00:00:00,1957759.0,1569.0,58.7,Medium
7124,WI,2023-05-10 00:00:00,2030717.0,1524.0,61.1,High
7125,WV,2023-05-10 00:00:00,650556.0,194.0,49.6,Medium


**Query 4: Window Function to Rank States by Weekly New Cases**

> **1. What is the goal of this query?**  
> To find out which states had the highest number of new COVID-19 cases most frequently across all weekly reports.

> **2. What does this query do?**  
> It uses the RANK() window function to assign a ranking to each state’s new case count per date, then filters to keep only the top-ranked (rank = 1) state for each date. It finally counts how many times each state appeared as the highest.

> **3. What does the result show?**  
> The output reveals that California (CA) had the most weeks ranked #1 in new COVID-19 cases (97 times), followed by Florida and Texas, highlighting states most frequently leading in infection spikes during the observed period.

In [17]:
query = """
-- QUERY 4: Rank states by new cases per date using window function, only extract rank 1
SELECT
    state,
    COUNT(date) AS num_weeks_rank1
FROM (
    SELECT
        state,
        date,
        new_cases,
        RANK() OVER (PARTITION BY date ORDER BY new_cases DESC) AS rank_by_cases
    FROM cases_deaths
)
WHERE rank_by_cases = 1
GROUP BY state
ORDER BY num_weeks_rank1 DESC, state;
"""

ranked_cases = pd.read_sql(query, conn) # Check unique states with rank 1
ranked_cases.to_csv("sql_results/query_4.csv", index=False) # Save to CSV for further analysis
ranked_cases

,state,num_weeks_rank1
0,CA,97
1,FL,25
2,TX,24
3,MI,8
4,IL,6
5,NYC,6
6,NJ,2
7,WA,2
8,DE,1
9,NY,1


**Query 5: Window Function to Determine National Week-over-Week % Change in New Cases**

> **1. What is the goal of this query?**  
> To calculate the percentage change in total new COVID-19 cases in the U.S. from one week to the next.

> **2. What does this query do?**  
> It aggregates new case totals across all states by week using a CTE, then applies the LAG() window function to fetch the prior week’s total and compute the week-over-week percent change.

> **3. What does the result show?**  
> The output reveals how national COVID-19 cases escalated dramatically in early 2020 (e.g., a 1021.78% spike in March) before stabilizing into regular fluctuations, highlighting key waves and transitions in the pandemic.

In [18]:
query = """
-- QUERY 5: National week-over-week % change in total new cases

WITH weekly_totals AS (
    SELECT
        date,
        SUM(new_cases) AS total_new_cases
    FROM cases_deaths
    GROUP BY date
),
weekly_change AS (
    SELECT
        date,
        total_new_cases,
        LAG(total_new_cases) OVER (ORDER BY date) AS previous_total_cases
    FROM weekly_totals
)
SELECT
    date,
    total_new_cases,
    previous_total_cases,
    ROUND(
        CASE 
            WHEN previous_total_cases = 0 THEN NULL
            ELSE ((total_new_cases - previous_total_cases) * 100.0 / previous_total_cases)
        END, 2
    ) AS percent_change
FROM weekly_change
ORDER BY date;
"""

national_wow_df = pd.read_sql(query, conn)
national_wow_df.to_csv("sql_results/query_5.csv", index=False)  # Save to CSV for further analysis
national_wow_df.head(20)


,date,total_new_cases,previous_total_cases,percent_change
0,2020-01-22 00:00:00,6.0,NaN,NaN
1,2020-01-29 00:00:00,5.0,6.0,-16.67
2,2020-02-05 00:00:00,12.0,5.0,140.00
3,2020-02-12 00:00:00,13.0,12.0,8.33
4,2020-02-19 00:00:00,16.0,13.0,23.08
5,2020-02-26 00:00:00,27.0,16.0,68.75
6,2020-03-04 00:00:00,101.0,27.0,274.07
7,2020-03-11 00:00:00,1133.0,101.0,1021.78
8,2020-03-18 00:00:00,7725.0,1133.0,581.82
9,2020-03-25 00:00:00,59286.0,7725.0,667.46


**Query 6: Window Function to Identify First Week with Reported COVID-19 Cases (> 0) Per State**

> **1. What is the goal of this query?**  
> To identify the first week each U.S. state recorded new COVID-19 cases (greater than zero), marking the onset of local outbreaks.

> **2. What does this query do?**  
> It filters the dataset to only include rows where new_cases > 0, then applies the ROW_NUMBER() window function partitioned by state to rank dates, keeping only the first recorded case per state.

> **3. What does the result show?**  
> The output reveals the earliest observed week of COVID-19 activity in each state (e.g., DE on Jan 22, 2020), giving insight into how the virus initially spread across different regions in the U.S.

In [19]:
query = """
-- QUERY 6: First week with > 0 new cases per state


WITH filtered_data AS (
    SELECT *
    FROM cases_deaths
    WHERE new_cases > 0
),
ranked_first_case AS (
    SELECT
        state,
        date,
        new_cases,
        ROW_NUMBER() OVER (PARTITION BY state ORDER BY date) AS rn
    FROM filtered_data
)
SELECT
    state,
    date AS first_reported_week,
    new_cases AS initial_reported_cases
FROM ranked_first_case
WHERE rn = 1
ORDER BY first_reported_week;
"""

first_nonzero_df = pd.read_sql(query, conn)
first_nonzero_df.to_csv("sql_results/query_6.csv", index=False)  # Save to CSV for further analysis
first_nonzero_df.head()

,state,first_reported_week,initial_reported_cases
0,DE,2020-01-22 00:00:00,5.0
1,WA,2020-01-22 00:00:00,1.0
2,AZ,2020-01-29 00:00:00,1.0
3,CA,2020-01-29 00:00:00,2.0
4,IL,2020-01-29 00:00:00,1.0


**Query 7: Group By State to Analyze Total and Average Weekly Cases, Deaths, and Vaccination Rates**

> **1. What is the goal of this query?**  
> To summarize the total and average weekly COVID-19 case counts, deaths, and vaccination rates for each U.S. state.

> **2. What does this query do?**  
> It joins the cases_deaths and vaccinations_weekly tables on state and date, then groups the data by state to calculate total new cases, total deaths, and the average weekly values for new cases, deaths, and vaccination percentages.

> **3. What does the result show?**  
> The output highlights state-level differences in COVID-19 impact. For example, California (CA) shows the highest total cases and deaths, while states vary significantly in their average booster and vaccination rates.

In [20]:
query = """
-- QUERY 7: Total and average weekly new cases and deaths per state

WITH merged_data AS (
    SELECT
        *
    FROM cases_deaths c
    JOIN vaccinations_weekly v
        ON c.state = v.state AND c.date = v.date
)

SELECT
    state,
    SUM(new_cases) AS total_cases,
    ROUND(AVG(new_cases), 2) AS avg_weekly_cases,
    SUM(new_deaths) AS total_deaths,
    ROUND(AVG(new_deaths), 2) AS avg_weekly_deaths,
    ROUND(AVG(booster_pct), 2) AS avg_booster_pct,
    ROUND(AVG(fully_vaccinated_pct), 2) AS avg_fully_vax_pct
FROM merged_data
GROUP BY state
ORDER BY total_cases DESC;
"""

impact_df = pd.read_sql(query, conn)
impact_df.to_csv("sql_results/query_7.csv", index=False)  # Save to CSV for further analysis
impact_df.head()

,state,total_cases,avg_weekly_cases,total_deaths,avg_weekly_deaths,avg_booster_pct,avg_fully_vax_pct
0,CA,10789983.0,85634.79,81633.0,647.88,32.18,57.54
1,TX,7146390.0,56717.38,69933.0,555.02,23.06,48.93
2,FL,6553754.0,52013.92,67666.0,537.03,25.12,53.90
3,IL,3324228.0,26382.76,27347.0,217.04,32.14,54.70
4,NY,3179953.0,25237.72,21760.0,172.70,27.62,62.02


**Query 8: Group By Booster Coverage Level to Compare Average Weekly Cases and Deaths**

> **1. What is the goal of this query?**  
> To compare how average weekly COVID-19 cases and deaths vary across different levels of booster vaccination coverage (Low, Medium, High).

> **2. What does this query do?**  
> It joins case and vaccination data, categorizes each record by booster coverage using a CASE statement, and then groups the data by coverage level to calculate average new cases and deaths for each group.

> **3. What does the result show?**  
> The results show that states with High booster coverage experienced significantly lower average new cases (2,740) and deaths (17.6) compared to Medium or Low groups, suggesting a strong protective impact of booster uptake.

In [21]:
query = """
-- QUERY 8: Group states by booster coverage and analyze average cases/deaths

WITH merged_data AS (
    SELECT
        c.state,
        c.new_cases,
        c.new_deaths,
        v.booster_pct
    FROM cases_deaths c
    JOIN vaccinations_weekly v
        ON c.state = v.state AND c.date = v.date
),
categorized_data AS (
    SELECT
        state,
        new_cases,
        new_deaths,
        booster_pct,
        CASE 
            WHEN booster_pct IS NULL THEN 'Unknown'
            WHEN booster_pct < 30 THEN 'Low'
            WHEN booster_pct < 60 THEN 'Medium'
            ELSE 'High'
        END AS booster_coverage_level
    FROM merged_data
)

SELECT
    booster_coverage_level,
    ROUND(AVG(new_cases), 2) AS avg_new_cases,
    ROUND(AVG(new_deaths), 2) AS avg_new_deaths,
    COUNT(*) AS weeks_count
FROM categorized_data
GROUP BY booster_coverage_level
ORDER BY avg_new_cases DESC;
"""

booster_impact_df = pd.read_sql(query, conn)
booster_impact_df.to_csv("sql_results/query_8.csv", index=False)  # Save to CSV for further analysis
booster_impact_df

,booster_coverage_level,avg_new_cases,avg_new_deaths,weeks_count
0,Medium,12838.26,79.80,3823
1,Low,11965.26,164.65,3049
2,High,2740.51,17.60,255


**Query 9: Subquery to Find States with Below-Average Vaccination and Above-Average Deaths**

> **1. What is the goal of this query?**  
> To identify states with low average vaccination rates and high average death rates — a potential red flag for COVID-19 vulnerability.

> **2. What does this query do?**  
> It calculates each state’s average vaccination and death rates using a GROUP BY, then filters results using two scalar subqueries to compare each state to the national average for vaccination and deaths.

> **3. What does the result show?**  
> It reveals that states like Texas, Georgia, and Ohio had significantly below-average vaccination rates and above-average weekly COVID-19 deaths, suggesting they were disproportionately impacted.

In [25]:
query = """
-- Query 9: States with below-average vaccination and above-average deaths

SELECT
    state,
    avg_vax,
    avg_deaths
FROM (
    SELECT 
        c.state,
        AVG(v.fully_vaccinated_pct) AS avg_vax,
        AVG(c.new_deaths) AS avg_deaths
    FROM vaccinations_weekly v
    JOIN cases_deaths c 
        ON v.state = c.state AND v.date = c.date
    GROUP BY c.state
) AS state_stats
WHERE 
    avg_vax < (SELECT AVG(fully_vaccinated_pct) FROM vaccinations_weekly)
    AND avg_deaths > (SELECT AVG(new_deaths) FROM cases_deaths)
ORDER BY avg_deaths DESC;
"""

high_risk_states_df = pd.read_sql(query, conn)
high_risk_states_df.to_csv("sql_results/query_9.csv", index=False)  # Save to CSV for further analysis
high_risk_states_df

,state,avg_vax,avg_deaths
0,TX,48.928571,555.023810
1,GA,43.902438,260.809524
2,OH,47.917347,258.357143
3,MI,49.427551,255.293651
4,AZ,49.990788,209.285714
5,TN,43.663832,193.468254
6,NC,49.944104,180.238095
7,IN,45.355442,152.039683
8,AL,40.862358,136.158730
9,KY,47.297732,130.992063


**Query 10: Subquery to Identify Top 5 States by Case Fatality Rate (CFR)**

> **1. What is the goal of this query?**  
> To find the five U.S. states with the highest overall COVID-19 case fatality rate — the percentage of reported cases that resulted in death.

> **2. What does this query do?**  
> It uses a subquery to calculate total cases and total deaths per state using GROUP BY, then computes the case fatality rate in the outer query by dividing deaths by cases and multiplying by 100. The query returns the top 5 states ordered by CFR.

> **3. What does the result show?**  
> States like Pennsylvania, Georgia, and Michigan had the highest fatality rates, with over 1.35% of reported cases resulting in death — potentially due to differences in healthcare access, demographics, or outbreak timing.

In [26]:
query = """
-- Query 10: Top 5 states by overall case fatality rate using a subquery

SELECT
    state,
    total_cases,
    total_deaths,
    ROUND(CAST(total_deaths AS FLOAT) * 100 / total_cases, 2) AS case_fatality_rate_pct
FROM (
    SELECT 
        state,
        SUM(new_cases) AS total_cases,
        SUM(new_deaths) AS total_deaths
    FROM cases_deaths
    GROUP BY state
) AS state_totals
WHERE total_cases > 0
ORDER BY case_fatality_rate_pct DESC
LIMIT 5;
"""

fatality_rate_df = pd.read_sql(query, conn)
fatality_rate_df.to_csv("sql_results/query_10.csv", index=False)  # Save to CSV for further analysis
fatality_rate_df

,state,total_cases,total_deaths,case_fatality_rate_pct
0,PA,3559331.0,51047.0,1.43
1,GA,3080929.0,42782.0,1.39
2,MI,3106362.0,42873.0,1.38
3,NYC,3266726.0,44624.0,1.37
4,AZ,2451004.0,33451.0,1.36


**Query 11: CTE to Identify Weeks with ≥50% Spike in National New COVID-19 Cases**

> **1. What is the goal of this query?**  
> To flag weeks where national COVID-19 cases increased by 50% or more compared to the previous week — acting as an alert system for potential surges.

> **2. What does this query do?**  
> It uses a CTE to calculate total weekly new cases, applies the LAG() window function to retrieve prior week counts, then filters for weeks where the percent change was ≥ 50%.

> **3. What does the result show?**  
> The output highlights key national rebound periods — including March 2020 and January 2022 — when COVID-19 case counts sharply spiked week-over-week, signaling new waves or outbreaks.

In [27]:
query = """
-- QUERY 11: COVID rebound alert — spikes of 50%+ in national new cases

WITH daily_totals AS (
    SELECT
        date,
        SUM(new_cases) AS total_new_cases
    FROM cases_deaths
    GROUP BY date
),
with_lagged AS (
    SELECT
        date,
        total_new_cases,
        LAG(total_new_cases) OVER (ORDER BY date) AS prev_week_cases
    FROM daily_totals
),
spike_alerts AS (
    SELECT
        date,
        total_new_cases,
        prev_week_cases,
        ROUND(((total_new_cases - prev_week_cases) * 100.0 / prev_week_cases), 2) AS percent_change
    FROM with_lagged
    WHERE prev_week_cases IS NOT NULL
)

SELECT *
FROM spike_alerts
WHERE percent_change >= 50
ORDER BY date;
"""

spike_alerts_df = pd.read_sql(query, conn)
spike_alerts_df.to_csv("sql_results/query_11.csv", index=False)  # Save to CSV for further analysis
spike_alerts_df

,date,total_new_cases,prev_week_cases,percent_change
0,2020-02-05 00:00:00,12.0,5.0,140.00
1,2020-02-26 00:00:00,27.0,16.0,68.75
2,2020-03-04 00:00:00,101.0,27.0,274.07
3,2020-03-11 00:00:00,1133.0,101.0,1021.78
4,2020-03-18 00:00:00,7725.0,1133.0,581.82
5,2020-03-25 00:00:00,59286.0,7725.0,667.46
6,2020-04-01 00:00:00,151264.0,59286.0,155.14
7,2021-07-14 00:00:00,251109.0,133430.0,88.20
8,2021-07-28 00:00:00,464744.0,288157.0,61.28
9,2021-12-29 00:00:00,2141043.0,1165714.0,83.67


**Query 12: Simulated Stored Procedure to Generate Weekly Summary for a Specific State**

> **1. What is the goal of this query?**  
> To create a reusable query that outputs weekly totals for cases, deaths, and average booster vaccination rate for a specific U.S. state based on input.

> **2. What does this query do?**  
> Since SQLite doesn't support stored procedures, it simulates one by allowing the user to substitute a state name (e.g., 'California') into a parameterized query. It joins the cases_deaths and vaccinations_weekly tables and groups results by date for the selected state.

> **3. What does the result show?**  
> It returns a weekly breakdown of new cases, deaths, and booster coverage for the specified state, enabling targeted trend analysis (e.g., how California’s metrics evolved over time).

```-- Simulated Stored Procedure Weekly COVID Summary for a State
-- This simulates a stored procedure that accepts a state name as input
-- For example: EXEC WeeklySummary('California')

BEGIN PROCEDURE
INPUT: state_name TEXT
OUTPUT: Weekly totals for cases, deaths, and avg booster rate

Parameter value (replace as needed)
SET @state_name = 'California';

SELECT
    c.date,
    c.state,
    SUM(c.new_cases) AS total_new_cases,
    SUM(c.new_deaths) AS total_new_deaths,
    ROUND(AVG(v.booster_pct), 2) AS avg_booster_pct
FROM cases_deaths c
JOIN vaccinations_weekly v
    ON c.state = v.state AND c.date = v.date
WHERE c.state = 'California' -- simulate: WHERE c.state = @state_name
GROUP BY c.date, c.state
ORDER BY c.date;
-- END PROCEDURE```